#A Computational Introduction to Number Theory (SM404) - **Assignment**:#

###**Team Members** : ###
1. Achintya Harsha (IMT2021525)
2. Devendra Rishi Nelapati (IMT2021076)

In [13]:
# installing necessary libraries
!pip install gmpy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
# importing required libraries
from random import randint
from math import floor
from gmpy2 import is_prime

###Class that has functions the implement Reed solomon error correction ###

In [20]:
class ReedSolomon():
    # initialization function
    def __init__(self):
        self.k = 0
        self.u = 0
        self.l = 0
        self.P = 1
        self.n = 1
        self.primes = []

    # egcd function that implements rational reconstruction
    def egcd(self, a, b):
        x0, x1, y0, y1 = 0, 1, 1, 0
        while a != 0:
            # loop breaks when a becomes less than M*P
            if(a < self.M * self.P):
                return a, y0
            q, b, a = b // a, a, b % a
            x0, x1 = x1 - q * x0, x0
            y0, y1 = y1 - q * y0, y0
        return a, y0

    def dBy2(self, a):
        return not(a % 2)

    # binary egcd implementation
    def binaryEGCD(self, a, b):
        r0, r1, e = a, b, 0
        s0, s1 = 1, 0
        t0, t1 = 0, 1

        while self.dBy2(r0) and self.dBy2(r1):
            r0, r1 = r0 >> 1, r1 >> 1
            e += 1

        aNew, bNew = r0, r1

        while r1 > 0:
            while self.dBy2(r0): 
                r0 = r0 >> 1
                if(self.dBy2(s0) and self.dBy2(t0)):
                    s0, t0 = s0 >> 1, t0 >> 1
                else:
                    s0, t0 = (s0 + bNew) >> 1, (t0 - aNew) >> 1

            while self.dBy2(r1): 
                r1 = r1 >> 1
                if(self.dBy2(s1) and self.dBy2(t1)):
                    s1, t1 = s1 >> 1, t1 >> 1
                else:
                    s1, t1 = (s1 + bNew) >> 1, (t1 - aNew) >> 1

            if(r1 < r0):
                r0, r1 = r1, r0
                s0, s1 = s1, s0
                t0, t1 = t1, t0
            r1 = r1 - r0
            s1 = s1 - s0
            t1 = t1 - t0

        return ((2**e)*r0, s0 , t0)

    # function that returns a prime number in a particular range
    def generatePrime(self, low, high):
        while True:
            prime = randint(low, high)
            if(prime in self.primes):
                continue
            if is_prime(prime):
                return prime

    # global setup function that sets all parameters for the Reed Solomon
    # error coreection model
    def golbalSetup(self, k, u, M):
        self.k = k # total number of codes
        self.u = u # corruption factor
        self.M = M # Upper bound of message input
        self.l = floor(k * u) # number of codes with errors

        # k primes are generated and stored in an array
        while(len(self.primes) < self.k):
            self.primes.append(self.generatePrime(10000, 50000))

        # primes are stroed in decreasing order
        self.primes.sort(reverse=True)

        print(f"Primes chosen n_k : {self.primes}")
        
        # P value is calculated
        # P = product of l largest primes
        for i in range(self.l):
            self.P *= self.primes[i]

        # product of all primes calculated
        for i in range(self.k):
            self.n *= self.primes[i]


    def crt(self, coefficients, primes):
        product = self.n
        result = 0
        idx = 0

        # crt calculated by using egcd to get the 
        # inverse of the number mod ni
        for coefficient in coefficients:
            ni = product // primes[idx]
            _, ni_inv, _ = self.binaryEGCD(ni, primes[idx])
            result += coefficient * ni * ni_inv
            idx += 1

        return result % product

    def transmit(self, ai):
        chosen = []
        bi = []

        # choosing error indexes
        while(len(chosen) < self.l):
            r = randint(0, self.k - 1)
            if(r not in chosen):
                chosen.append(r)

        # introducing errors in transmission codes
        # at l indexes randomly chosen
        print(f"Chosen error indexes : {chosen}")
        i = 0
        while(i < self.k):
            if(i in chosen):
                bi.append(randint(100, 500))
            else:
                bi.append(ai[i])
            i += 1
        
        print(f"Transmitted codes with errors : {bi}")
        return bi


    def ReedSolomonSend(self, a):
        ai = []

        # codes are generated by taking mod of the input message
        # with the k chosen primes
        for i in range(self.k):
            ai.append(a % self.primes[i])
        print(f"a mod ni array : {ai}")
        return self.transmit(ai)

    def ReedSolomonReceive(self, bi):
        # b value calculated using crt between bi's(the received codes)
        # and the k chosen primes
        b = self.crt(bi, self.primes)

        # r' and t' are calculated using rational reconstruction
        rDash, tDash = self.egcd(b, self.n)

        # the orignal message is returned if reconctruction is possible
        if(tDash == 0):
            return rDash
        elif(rDash % tDash == 0):
            return rDash // tDash
        else:
            # else errore is returned
            return "Error"

###Testing Reed Solomon Error correction###

In [ ]:
k = int(input("Enter number of codes (number of primes) : "))
u = float(input("Corruption percenatge - (0 - 1) : "))
M = 10**20

reedSol = ReedSolomon()
reedSol.golbalSetup(k, u, M)

while True:
    msg = int(input("\nEnter the message(Integer) you wish to send :"))
    
    print("\n********************************************************************************************************")
    rcvMsg = reedSol.ReedSolomonSend(msg)
    
    decodedMsg = reedSol.ReedSolomonReceive(rcvMsg)
    print("********************************************************************************************************\n")

    print(f"Received message : {decodedMsg}\n################\n")